In [ ]:
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt
import pandas as pd
import scikit_posthocs as sp
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import scipy.stats as stats
from random import randint, uniform
from scipy.stats import randint as sp_randint, uniform
import warnings
import numpy as np 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import joblib  
import os  
import plotly.express as px
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold  # Import StratifiedKFold
from sklearn.feature_selection import RFECV



# --- Variable Encoding Function ---
def encode_variables(df):  
    """
    Encodes the DataFrame variables according to the specified plan.

    Args:
        df: pandas DataFrame with the original data.

    Returns:
        DataFrame: DataFrame with encoded variables.
    """

    df_encoded = df.copy()

    # 1. anxiety_level: Ordinal Encoding (0, 1, 2, 3)
    anxiety_mapping = {
        "minimal": 0,
        "mild": 1,
        "moderate": 2,
        "severe": 3,
    }
    df_encoded["anxiety_level"] = df_encoded["anxiety_level"].map(anxiety_mapping)

    # 2. sex: Binary Encoding (0 and 1)
    sex_mapping = {"female": 0, "male": 1}
    df_encoded["sex"] = df_encoded["sex"].map(sex_mapping)

    # 3. education_level: Ordinal Encoding (0, 1, 2)
    education_mapping = {
        "technical level": 0,
        "bachelor": 1,
        "graduate": 2,
    }
    df_encoded["education_level"] = df_encoded["education_level"].map(
        education_mapping
    )

    # 4. shift: One-Hot Encoding
    df_encoded = pd.get_dummies(
        df_encoded, columns=["shift"], prefix="shift", dummy_na=False
    )

    # 5. marital_status: One-Hot Encoding + Handling 'widowed'
    df_encoded["marital_status"] = df_encoded["marital_status"].replace(
        "widowed", "single"
    )  # Group with 'single'
    df_encoded = pd.get_dummies(
        df_encoded, columns=["marital_status"], prefix="marital", dummy_na=False
    )

    # 6. category: One-Hot Encoding + Grouping
    df_encoded["category"] = df_encoded["category"].replace(
        ["spec nurse", "head nurse"], "other"
    )  # Group
    df_encoded = pd.get_dummies(
        df_encoded, columns=["category"], prefix="category", dummy_na=False
    )

    # 7. age_range: Ordinal Encoding
    age_categories = ["20 to 29", "30 to 39", "40 to 49", "50 and over"]
    age_encoder = OrdinalEncoder(categories=[age_categories])
    df_encoded["age_range"] = age_encoder.fit_transform(
        df_encoded[["age_range"]]
    )
    df_encoded["age_range"] = df_encoded["age_range"].astype(int)

    # 8. seniority_range: Ordinal Encoding
    seniority_categories = [
        "1 to 5",
        "6 to 10",
        "11 to 15",
        "16 to 20",
        "21 and over",
    ]
    seniority_encoder = OrdinalEncoder(categories=[seniority_categories])
    df_encoded["seniority_range"] = seniority_encoder.fit_transform(
        df_encoded[["seniority_range"]]
    )
    df_encoded["seniority_range"] = df_encoded["seniority_range"].astype(int)

  # --- CREATION OF INTERACTION VARIABLES ---

    # 1. Interactions between numerical/ordinal variables:
    df_encoded['age_x_seniority'] = df_encoded['age_range'] * df_encoded['seniority_range']
    df_encoded['age_x_education'] = df_encoded['age_range'] * df_encoded['education_level']
    df_encoded['seniority_x_education'] = df_encoded['seniority_range'] * df_encoded['education_level']

    # 2. Interactions between sex and other variables:
    df_encoded['sex_x_age'] = df_encoded['sex'] * df_encoded['age_range']
    df_encoded['sex_x_seniority'] = df_encoded['sex'] * df_encoded['seniority_range']
    df_encoded['sex_x_education'] = df_encoded['sex'] * df_encoded['education_level']

    # 3. Interactions between one-hot encoded variables and numerical/ordinal variables:
    for col in ['shift_afternoon', 'shift_morning', 'shift_night a', 'shift_night b',
                'marital_domestic partnership', 'marital_married', 'marital_single',
                'category_gen nurse', 'category_nurse aux', 'category_other']:
        df_encoded[f'{col}_x_age'] = df_encoded[col] * df_encoded['age_range']
        df_encoded[f'{col}_x_seniority'] = df_encoded[col] * df_encoded['seniority_range']
        df_encoded[f'{col}_x_education'] = df_encoded[col] * df_encoded['education_level']


    # 4. Interactions between one-hot encoded variables (categorical):
    for col1 in ['shift_afternoon', 'shift_morning', 'shift_night a', 'shift_night b']:
        for col2 in ['marital_domestic partnership', 'marital_married', 'marital_single']:
            df_encoded[f'{col1}_x_{col2}'] = df_encoded[col1] * df_encoded[col2]

    for col1 in ['shift_afternoon', 'shift_morning', 'shift_night a', 'shift_night b']:
        for col2 in ['category_gen nurse', 'category_nurse aux', 'category_other']:
            df_encoded[f'{col1}_x_{col2}'] = df_encoded[col1] * df_encoded[col2]

    for col1 in ['marital_domestic partnership', 'marital_married', 'marital_single']:
        for col2 in ['category_gen nurse', 'category_nurse aux', 'category_other']:
            df_encoded[f'{col1}_x_{col2}'] = df_encoded[col1] * df_encoded[col2]

    return df_encoded



# --- Data Augmentation Function with SMOTE ---
def augment_data_smote(
    df, target_variable, sampling_strategy="auto", random_state=None, k_neighbors=None
):
    """
    Applies SMOTE to oversample the target variable, with improved error handling
    and k_neighbors adjustment.

    Args:
        df: pandas DataFrame with the encoded data.
        target_variable: Name of the column containing the (categorical) target variable.
        sampling_strategy: Sampling strategy.
        random_state: Random seed.
        k_neighbors: Number of neighbors.

    Returns:
        DataFrame, Series: DataFrame with augmented predictor variables and Series with the
                         augmented target variable.  Or None, None if there's an error.
    """

    X = df.drop(target_variable, axis=1)
    y = df[target_variable]

    if not pd.api.types.is_numeric_dtype(y):
        print("Error: The target variable must be numeric (ordinally encoded).")
        return None, None

    if len(y.unique()) < 2:
        print("Error: The target variable must have at least two different classes.")
        return None, None

    class_counts = Counter(y)
    min_samples = min(class_counts.values())

    if k_neighbors is None:
        k_neighbors = min(5, min_samples - 1)
    else:
        k_neighbors = min(k_neighbors, min_samples - 1)

    if k_neighbors < 1:
        print("Error: At least one class has very few samples. SMOTE cannot be applied.")
        print("Consider grouping minority classes or removing the solitary sample.")
        return None, None

    if min_samples <= 1:
        print("Error: At least one class has only one sample. SMOTE cannot be applied.")
        print("Consider grouping minority classes or removing the solitary sample before applying SMOTE")
        return None, None

    print(f"Using k_neighbors = {k_neighbors} for SMOTE.")

    try:
        smote = SMOTE(
            sampling_strategy=sampling_strategy,
            random_state=random_state,
            k_neighbors=k_neighbors,
        )
        X_resampled, y_resampled = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error applying SMOTE: {e}")
        return None, None

    print("Class distribution before SMOTE:", Counter(y))
    print("Class distribution after SMOTE:", Counter(y_resampled))

    return X_resampled, y_resampled


def analyze_spearman_correlation(df, target_variable):
    """
    Calculates and visualizes Spearman correlations between the (ordinal) target variable
    and other numerical/ordinal variables in the DataFrame.

    Args:
        df: pandas DataFrame with the data (already augmented and encoded).
        target_variable: Name of the column containing the (ordinal) target variable.

    Returns:
        None (prints the correlation matrix and shows a heatmap).
    """

    # Calculate the Spearman correlation matrix
    spearman_correlations = df.corr(method="spearman")

    # Extract correlations with the target variable
    correlations_with_target = spearman_correlations[target_variable].drop(
        target_variable
    )  # Exclude self-correlation

    # Print correlations with the target variable
    print("Spearman Correlations with", target_variable, ":\n")
    print(correlations_with_target.sort_values(ascending=False))

    # Visualize with a heatmap (all variables)
    plt.figure(figsize=(12, 10))
    sns.heatmap(spearman_correlations, annot=True, cmap="coolwarm", center=0)
    plt.title("Spearman Correlation Heatmap")
    plt.tight_layout()
    plt.savefig("spearman_correlation_heatmap.png")  # Save the figure
    plt.close() #Close figure

    # Visualize with a bar plot (only correlations with the target variable)
    plt.figure(figsize=(8, 6))
    correlations_with_target.sort_values().plot(kind="barh", color="skyblue")
    plt.title(f"Spearman Correlation with {target_variable}")
    plt.xlabel("Spearman Correlation Coefficient")
    plt.ylabel("Variables")
    plt.tight_layout()
    plt.savefig("spearman_correlation_barplot.png")  # Save the figure
    plt.close()


def inferential_analysis(df, target_variable):
    """
    Performs Kruskal-Wallis and Chi-square tests to analyze relationships
    between the target variable and other variables in the DataFrame.

    Args:
        df: pandas DataFrame with the data (already augmented and encoded).
        target_variable: Name of the column containing the (ordinal) target variable.

    Returns:
        None (prints the test results).
    """

    # --- 1. Kruskal-Wallis (for numerical/ordinal variables) ---
    print("-" * 50)
    print("KRUSKAL-WALLIS TESTS")
    print("-" * 50)

    # List of numerical/ordinal variables (excluding target and one-hot encoded)
    numerical_ordinal_vars = [
        col
        for col in df.columns
        if (
            pd.api.types.is_numeric_dtype(df[col])
            and col != target_variable
            and not col.startswith(("shift_", "marital_", "category_"))
        )
    ]

    for variable in numerical_ordinal_vars:
        # Group data by levels of the target variable
        groups = [
            df[variable][df[target_variable] == level]
            for level in df[target_variable].unique()
        ]

        # Check for at least two groups and non-empty groups
        if len(groups) < 2:
            print(
                f"Cannot perform Kruskal-Wallis on {variable}: fewer than two groups."
            )
            continue
        if any(len(group) == 0 for group in groups):
            print(
                f"Cannot perform Kruskal-Wallis on {variable}: at least one group is empty."
            )
            continue

        # Perform Kruskal-Wallis test
        try:
            statistic, p_value = stats.kruskal(*groups)
        except ValueError as e:
            print(f"Error performing Kruskal-Wallis on {variable}: {e}")
            print("This may occur if all values in a group are equal.")
            continue

        print(f"\nKruskal-Wallis: {variable} vs. {target_variable}")
        print(f"  H Statistic: {statistic:.3f}")
        print(f"  p-value: {p_value:.3f}")

        # Interpretation
        if p_value < 0.05:
            print(
                f"  Result: Significant differences between levels of {target_variable} on variable {variable}."
            )

            # --- Post-Hoc Tests (Dunn with Bonferroni/Holm correction) ---
            # Dunn's test
            dunn_result = sp.posthoc_dunn(
                a=df,
                val_col=variable,
                group_col=target_variable,
                p_adjust="bonferroni",  # Or 'holm'
            )
            print("\n  Dunn's Test (Post-Hoc):")
            print(dunn_result)

        else:
            print(
                f"  Result: No significant differences between levels of {target_variable} on variable {variable}."
            )

    # --- 2. Chi-square (for categorical variables) ---
    print("\n" + "-" * 50)
    print("CHI-SQUARE TESTS")
    print("-" * 50)

    # List of categorical variables (including one-hot encoded)
    categorical_vars = [
        col
        for col in df.columns
        if col
        in (
            "sex",
            "shift_afternoon",
            "shift_morning",
            "shift_night a",
            "shift_night b",
            "marital_domestic partnership",
            "marital_married",
            "marital_single",
            "category_gen nurse",
            "category_nurse aux",
            "category_other",
            "education_level"
        )
    ]

    for variable in categorical_vars:
        # Create contingency table
        contingency_table = pd.crosstab(df[target_variable], df[variable])
        print(f"\nContingency Table: {target_variable} vs. {variable}")
        print(contingency_table)

        # Perform Chi-square test
        try:
            chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
            print(f"\nChi-square: {variable} vs. {target_variable}")
            print(f"  Chi2 Statistic: {chi2:.3f}")
            print(f"  p-value: {p_value:.3f}")
            print(f"  Degrees of freedom: {dof}")

            # Interpretation
            if p_value < 0.05:
                print(
                    f"  Result: Significant association between {target_variable} and {variable}."
                )
                # Calculate Cramer's V
                n = contingency_table.sum().sum()
                phi2 = chi2 / n
                r, k = contingency_table.shape
                phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
                rcorr = r - ((r - 1) ** 2) / (n - 1)
                kcorr = k - ((k - 1) ** 2) / (n - 1)
                cramers_v = (phi2corr / min((kcorr - 1), (rcorr - 1))) ** 0.5
                print(f"  Cramer's V: {cramers_v:.3f}")

            else:
                print(
                    f"  Result: No significant association between {target_variable} and {variable}."
                )

        except ValueError as e:
            print(f"Error performing Chi-square on {variable}: {e}")
            print("This may occur if any expected frequencies are zero.")
            continue

def train_model(model, X_train, y_train):
    """Trains a given model."""
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluates a model and prints metrics.  Returns test accuracy."""
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    conf_matrix = confusion_matrix(y_test, model.predict(X_test))

    # Handle multiclass classification report
    try:
        class_report = classification_report(y_test, model.predict(X_test))
    except ValueError as e:
        print(f"Error generating classification report: {e}")
        print("This usually happens if the predicted labels don't cover all classes present in y_test.")
        print("Check if your model is predicting all classes, especially after SMOTE and train/test split.")
        class_report = "Could not generate report (see error above)"

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}\n")
    print(f"Confusion Matrix:\n{conf_matrix}\n")
    print(f"Classification Report:\n{class_report}")
    return test_accuracy



def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluates a model and prints metrics.  Returns test accuracy."""
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    conf_matrix = confusion_matrix(y_test, model.predict(X_test))

    # Handle multiclass classification report
    try:
        class_report = classification_report(y_test, model.predict(X_test))
    except ValueError as e:
        print(f"Error generating classification report: {e}")
        print("This usually happens if the predicted labels don't cover all classes present in y_test.")
        print("Check if your model is predicting all classes, especially after SMOTE and train/test split.")
        class_report = "Could not generate report (see error above)"

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}\n")
    print(f"Confusion Matrix:\n{conf_matrix}\n")
    print(f"Classification Report:\n{class_report}")
    return test_accuracy



def optimize_and_evaluate(model_name, model, param_dist, X_train, y_train, X_test, y_test, n_iter=20, cv=5):
    """Performs RandomizedSearchCV, saves/loads results, evaluates, and returns test accuracy."""

    save_path = f"{model_name}_random_search.pkl"  # Simplified path

    try:
        if os.path.exists(save_path):
            random_search = joblib.load(save_path)
            print(f"Loaded RandomizedSearchCV for {model_name} from {save_path}")
        else:
            print(f"No checkpoint found for {model_name}. Starting RandomizedSearchCV.")
            print(f"Type of param_dist: {type(param_dist)}")  # Debug: Check type
            print(f"Contents of param_dist: {param_dist}") # Debug: Check contents

            random_search = RandomizedSearchCV(
                model,
                param_dist,
                n_iter=n_iter,
                cv=cv,
                scoring='accuracy',
                n_jobs=-1,
                random_state=42,
                verbose=0
            )
            random_search.fit(X_train, y_train)
            joblib.dump(random_search, save_path)  # Save after fitting
            print(f"RandomizedSearchCV for {model_name} saved to {save_path}")


        best_model = random_search.best_estimator_
        print(f"Best parameters for {model_name}: {random_search.best_params_}")
        test_accuracy = evaluate_model(best_model, X_train, y_train, X_test, y_test)
        return test_accuracy

    except Exception as e:
        print(f"Error in optimize_and_evaluate for {model_name}: {e}")
        return None


def create_models_dict():
    """Creates the dictionary of models with pipelines for scaling."""
    models = {
        "KNN": Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "Gradient Boosting": Pipeline([('scaler', StandardScaler()), ('gb', GradientBoostingClassifier(random_state=42))]),
        "XGBoost": Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))]), #Multiclass classification
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }
    return models


def create_param_distributions():
    """Defines parameter distributions for RandomizedSearchCV."""
    param_distributions = {
        "KNN": {
            'knn__n_neighbors': sp_randint(3, 200),  
            'knn__weights': ['uniform', 'distance'],
            'knn__p': [1, 2]
        },
        "Decision Tree": {
            'max_depth': sp_randint(3, 200),  
            'min_samples_split': sp_randint(2, 200), 
            'min_samples_leaf': sp_randint(1, 200),  
            'criterion': ['gini', 'entropy', "log_loss"]
        },
        "Random Forest": {
            'n_estimators': sp_randint(100, 5001), 
            'max_depth': sp_randint(3, 31),
            'min_samples_split': sp_randint(2, 41),
            'min_samples_leaf': sp_randint(1, 41),
            'max_features': [None, 'sqrt', 'log2'],
            'bootstrap': [True, False]
        },
        "AdaBoost": {
            'n_estimators': sp_randint(50, 1001), 
            'learning_rate': uniform(0.001, 0.9)  
        },
        "Gradient Boosting": {
            'gb__n_estimators': sp_randint(100, 5001), 
            'gb__learning_rate': uniform(0.001, 0.5),  
            'gb__max_depth': sp_randint(3, 64), 
            'gb__min_samples_split': sp_randint(2, 21),
            'gb__min_samples_leaf': sp_randint(1, 21),
            'gb__subsample': uniform(0.6, 0.8),  
            'gb__max_features': [None, 'sqrt', 'log2']
        },
        "XGBoost": {
            'xgb__n_estimators': sp_randint(100, 5001), 
            'xgb__learning_rate': uniform(0.001, 0.6),  
            'xgb__max_depth': sp_randint(3, 64), 
            'xgb__subsample': uniform(0.6, 0.2),  
            'xgb__colsample_bytree': uniform(0.6, 0.2),
            'xgb__gamma': uniform(0, 0.5)
        },
        "Extra Trees": {
            'n_estimators': sp_randint(100, 5001), 
            'max_depth': sp_randint(3, 31),
            'min_samples_split': sp_randint(2, 41),
            'min_samples_leaf': sp_randint(1, 41),
            'max_features': [None, 'sqrt', 'log2'],
            'bootstrap': [True, False]
        }
    }
    return param_distributions



## Exploratory Data Analysis and Statistical Significance Testing

In [ ]:
# 1. Load Data
df = pd.read_csv(r"D:\ansiedad\AnxietyLevelByCovid.csv")

In [116]:
# 2. Encode Variables
df_encoded = encode_variables(df)


In [117]:
# 3. Augment Data
sampling_strategy = {
    0: 100,  # minimal
    1: 100,  # mild
    2: 100,  # moderate
    3: 100,  # severe
}

X_resampled, y_resampled = augment_data_smote(
    df_encoded,
    target_variable="anxiety_level",
    sampling_strategy=sampling_strategy,
    random_state=42,
    #k_neighbors=3,  # Optional
)

Using k_neighbors = 5 for SMOTE.
Class distribution before SMOTE: Counter({0: 62, 1: 49, 2: 22, 3: 7})
Class distribution after SMOTE: Counter({1: 100, 0: 100, 2: 100, 3: 100})


In [118]:
# 4. Create Augmented DataFrame
if X_resampled is not None and y_resampled is not None:
    df_augmented = pd.DataFrame(X_resampled, columns=X_resampled.columns)
    df_augmented["anxiety_level"] = y_resampled

    # --- Now work with df_augmented ---
    print("\nAugmented DataFrame:")
    print(df_augmented.head())
    print(df_augmented.shape)

    # --- Perform correlation analysis, inferential analysis, etc. ---
    analyze_spearman_correlation(df_augmented, "anxiety_level")
    inferential_analysis(df_augmented, "anxiety_level")


else:
    print("Data augmentation failed. Check for errors.")


Augmented DataFrame:
   sex  education_level  age_range  seniority_range  shift_afternoon  \
0    0                1          0                0            False   
1    0                2          2                3            False   
2    1                2          1                2            False   
3    0                1          2                3            False   
4    0                2          1                2            False   

   shift_morning  shift_night a  shift_night b  marital_domestic partnership  \
0          False           True          False                          True   
1           True          False          False                         False   
2           True          False          False                         False   
3           True          False          False                          True   
4           True          False          False                         False   

   marital_married  ...  marital_domestic partnership_x_category

In [119]:
df_augmented.shape

(400, 84)

In [127]:
analyze_spearman_correlation(df_augmented, "anxiety_level")

Spearman Correlations with anxiety_level :

education_level                                  0.063957
shift_night b_x_education                        0.053155
shift_night b_x_seniority                        0.021915
marital_single_x_education                       0.010306
marital_domestic partnership_x_category_other    0.000000
                                                   ...   
shift_afternoon_x_age                           -0.223731
age_range                                       -0.233366
shift_afternoon_x_marital_single                -0.242115
age_x_seniority                                 -0.244353
seniority_range                                 -0.267716
Name: anxiety_level, Length: 83, dtype: float64


## Spearman Correlation Analysis Summary

This section summarizes the findings from the Spearman rank correlation analysis, focusing on the relationships between the `anxiety_level` (our ordinal target variable) and other predictor variables in the dataset (after data augmentation with SMOTE and feature engineering).

**Key Findings:**

*   **Generally Weak Correlations:** Overall, most predictor variables exhibit weak correlations with `anxiety_level`, with the strongest correlation being -0.267716.  This reinforces that anxiety, as measured in this study, is a complex phenomenon and is likely influenced by a combination of factors, rather than any single strong predictor.

*   **Negative Correlations (Weak):**
    *   `seniority_range` (-0.267716): Shows the strongest negative correlation, suggesting that nurses with *more* experience tend to report *lower* anxiety levels.
    *   `age_x_seniority` (-0.244353): An interaction term between age and seniority. The negative correlation suggests the combined effect of higher age and more seniority is associated with lower reported anxiety. This may largely reflect the `seniority_range` effect, as age and seniority are likely correlated.
    *   `shift_afternoon_x_marital_single` (-0.242115):  An interaction term. Nurses who work the afternoon shift *and* are single tend to report slightly lower anxiety.
    *  `age_range` (-0.233366):  Older nurses *tend* to report slightly lower anxiety levels.
    *  `shift_afternoon_x_age` (-0.223731):  Nurses who work the afternoon shift and are in older age range *tend* to report slightly lower anxiety levels.

*   **Positive Correlations (Very Weak):**
    *   `education_level` (0.063957):  A very weak positive correlation, suggesting a slight, and potentially spurious, tendency for nurses with higher education levels to report *higher* anxiety.  Further investigation (hypothesis testing) is needed.
    *   `shift_night b_x_education` (0.053155): A very weak positive correlation. Nurses working the "night b" shift *and* having higher education levels tend to report *slightly* higher anxiety.
    *  `shift_night b_x_seniority` (0.021915) and `marital_single_x_education` (0.010306) show correlations very close to zero.

*   **Multicollinearity:** While not shown in this specific output, remember that strong correlations *between predictor variables* (like `age_range` and `seniority_range`) indicate multicollinearity. This should be addressed in modeling (e.g., by using only one of the highly correlated variables, or creating a combined variable).

* **Variables with very weak correlation (near zero)**
    * Many variables, and specially interactions, show correlation very close to zero.


In [121]:
inferential_analysis(df_augmented, "anxiety_level")

--------------------------------------------------
KRUSKAL-WALLIS TESTS
--------------------------------------------------

Kruskal-Wallis: sex vs. anxiety_level
  H Statistic: 16.436
  p-value: 0.001
  Result: Significant differences between levels of anxiety_level on variable sex.

  Dunn's Test (Post-Hoc):
          0         1         2         3
0  1.000000  0.302845  0.302845  1.000000
1  0.302845  1.000000  0.000550  0.028349
2  0.302845  0.000550  1.000000  1.000000
3  1.000000  0.028349  1.000000  1.000000

Kruskal-Wallis: education_level vs. anxiety_level
  H Statistic: 2.777
  p-value: 0.427
  Result: No significant differences between levels of anxiety_level on variable education_level.

Kruskal-Wallis: age_range vs. anxiety_level
  H Statistic: 28.961
  p-value: 0.000
  Result: Significant differences between levels of anxiety_level on variable age_range.

  Dunn's Test (Post-Hoc):
         0         1         2         3
0  1.00000  1.000000  1.000000  0.000050
1  1.00000

## Statistical Analysis: Key Conclusions

This section summarizes the key findings from the statistical analysis, building upon the exploratory data analysis, correlation analysis, and incorporating results from the inferential tests *after including interaction terms*. We used non-parametric hypothesis tests (Kruskal-Wallis and Chi-square) to assess the statistical significance of relationships between the ordinal `anxiety_level` variable and other predictors, including interaction terms.

**Main Conclusions:**

*   **Significant Associations with Anxiety:** Several demographic, work-related factors, *and interaction terms*, show statistically significant associations with nurses' anxiety levels during the COVID-19 pandemic (p < 0.05, after appropriate corrections for multiple comparisons). These factors include:

    *   **Original Variables:**
        *   `sex` (Kruskal-Wallis and Chi-square)
        *   `age_range` (Kruskal-Wallis)
        *   `seniority_range` (Kruskal-Wallis)
        *   All `shift` variables (Chi-square: `shift_afternoon`, `shift_morning`, `shift_night a`, `shift_night b`)
        *   All `marital_status` variables (Chi-square: `marital_domestic partnership`, `marital_married`, `marital_single`)
        *   All `category` variables (Chi-square: `category_gen nurse`, `category_nurse aux`, `category_other`)

    *   **Interaction Terms (Kruskal-Wallis):**
        *   `age_x_seniority`
        *   `age_x_education`
        *   `seniority_x_education`
        *   `sex_x_age`
        *   `sex_x_seniority`
        *   `sex_x_education`

*   **Non-Significant Association with Education Level (Alone):**  `education_level`, *when considered independently*, did *not* show a statistically significant association with anxiety levels in either the Kruskal-Wallis or Chi-square tests.  This is consistent with the findings before the inclusion of interaction terms.  *However*, interaction terms *involving* education *do* show significant associations (see above).

*   **Strength of Associations:** While many associations were statistically significant, it's crucial to remember that the Spearman correlation analysis generally showed *weak* correlations, and the Cramer's V values (from the Chi-square tests) were mostly in the low to moderate range.  This indicates that, while these factors are *related* to anxiety, no single factor is a *strong* predictor on its own. Anxiety is a complex, multi-factorial outcome.

* **Importance of Interactions:** The significance of several interaction terms in the Kruskal-Wallis tests highlights that the relationship between these factors and anxiety is not always simple or additive.  The effect of one variable (e.g., age) on anxiety *depends* on the level of another variable (e.g., seniority or education).  This justifies the inclusion of interaction terms in the subsequent predictive modeling.


In [ ]:
if __name__ == "__main__":
   
    # --- FEATURE ENGINEERING 
    df_encoded = encode_variables(df)  

    # --- SMOTE and train_test_split AFTER feature engineering ---
    X = df_encoded.drop('anxiety_level', axis=1)
    y = df_encoded['anxiety_level']

    X_resampled, y_resampled = augment_data_smote(df_encoded, 'anxiety_level', sampling_strategy={0: 100, 1: 100, 2: 100, 3: 100}, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

   
    # 3. Create models and parameter distributions
    models = create_models_dict()
    param_distributions = create_param_distributions()

    # --- Model Optimization and Evaluation Loop ---
    results = {}
    for name, model in models.items():
        print(f"--- Optimizing and Evaluating {name} ---")
        if name in param_distributions:
            # Train and evaluate using the FULL feature set (X_train, X_test)
            test_accuracy = optimize_and_evaluate(name, model, param_distributions[name], X_train, y_train, X_test, y_test, n_iter= 50) # Aumentamos el numero de iteraciones
            if test_accuracy is not None:
                results[name] = test_accuracy
        else:
            print(f"No parameter distributions found for {name}.  Training without optimization.")
            trained_model = train_model(model, X_train, y_train)
            test_accuracy = evaluate_model(trained_model, X_train, y_train, X_test, y_test)
            results[name] = test_accuracy

     # --- Model Comparison (with Plotly) ---
    print("\n--- Model Comparison ---")
    results_df = pd.DataFrame(list(results.items()), columns=['Model', 'Test Accuracy'])
    results_df = results_df.sort_values(by='Test Accuracy', ascending=False)  # Sort before plotting
    print(results_df)

    # Create the bar chart with Plotly
    fig = px.bar(results_df, x='Test Accuracy', y='Model',
                title='Model Comparison (Test Accuracy)',
                orientation='h',  # Horizontal bar chart
                color='Model',  # Color bars by model
                )
    fig.update_layout(yaxis={'categoryorder':'total ascending'}) #Sort
    fig.show()

Using k_neighbors = 5 for SMOTE.
Class distribution before SMOTE: Counter({0: 62, 1: 49, 2: 22, 3: 7})
Class distribution after SMOTE: Counter({1: 100, 0: 100, 2: 100, 3: 100})
--- Optimizing and Evaluating KNN ---
Loaded RandomizedSearchCV for KNN from KNN_random_search.pkl
Best parameters for KNN: {'knn__n_neighbors': 7, 'knn__p': 2, 'knn__weights': 'uniform'}
Training Accuracy: 0.7031
Test Accuracy: 0.6625

Confusion Matrix:
[[13  4  2  1]
 [ 4 11  1  4]
 [ 1  3 12  4]
 [ 1  0  2 17]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.65      0.67        20
           1       0.61      0.55      0.58        20
           2       0.71      0.60      0.65        20
           3       0.65      0.85      0.74        20

    accuracy                           0.66        80
   macro avg       0.66      0.66      0.66        80
weighted avg       0.66      0.66      0.66        80

--- Optimizing and Evaluating Decision Tree ---


C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning:


135 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

RandomizedSearchCV for Gradient Boosting saved to Gradient Boosting_random_search.pkl
Best parameters for Gradient Boosting: {'gb__learning_rate': np.float64(0.011292247147901224), 'gb__max_depth': 4, 'gb__max_features': 'sqrt', 'gb__min_samples_leaf': 6, 'gb__min_samples_split': 3, 'gb__n_estimators': 1284, 'gb__subsample': np.float64(0.8433937943676302)}
Training Accuracy: 0.8719
Test Accuracy: 0.7250

Confusion Matrix:
[[17  2  1  0]
 [ 5 10  3  2]
 [ 2  4 13  1]
 [ 1  0  1 18]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.85      0.76        20
           1       0.62      0.50      0.56        20
           2       0.72      0.65      0.68        20
           3       0.86      0.90      0.88        20

    accuracy                           0.72        80
   macro avg       0.72      0.72      0.72        80
weighted avg       0.72      0.72      0.72        80

--- Optimizing and Evaluating XGBoost ---
No checkpoin

C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning:

[09:33:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




RandomizedSearchCV for XGBoost saved to XGBoost_random_search.pkl
Best parameters for XGBoost: {'xgb__colsample_bytree': np.float64(0.7670604991178476), 'xgb__gamma': np.float64(0.16039003248586792), 'xgb__learning_rate': np.float64(0.11291110623991253), 'xgb__max_depth': 31, 'xgb__n_estimators': 1894, 'xgb__subsample': np.float64(0.7181785886376484)}
Training Accuracy: 0.9000
Test Accuracy: 0.7125

Confusion Matrix:
[[16  2  2  0]
 [ 6 11  2  1]
 [ 5  1 13  1]
 [ 2  0  1 17]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.80      0.65        20
           1       0.79      0.55      0.65        20
           2       0.72      0.65      0.68        20
           3       0.89      0.85      0.87        20

    accuracy                           0.71        80
   macro avg       0.74      0.71      0.71        80
weighted avg       0.74      0.71      0.71        80

--- Optimizing and Evaluating Extra Trees ---
No checkpoint

In [126]:
if __name__ == "__main__":
  
    df_encoded = encode_variables(df)
    X = df_encoded.drop('anxiety_level', axis=1)
    y = df_encoded['anxiety_level']
    X_resampled, y_resampled = augment_data_smote(df_encoded, 'anxiety_level',
                                                sampling_strategy={0: 100, 1: 100, 2: 100, 3: 100},
                                                random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,
                                                    test_size=0.2, random_state=42,
                                                    stratify=y_resampled)

    # --- Train Gradient Boosting (with the best hyperparameters) ---
    best_gb_params = {
        'n_estimators': 1284,
        'learning_rate': 0.011292247147901224,
        'max_depth': 4,
        'subsample': 0.8433937943676302,
        'max_features': 'sqrt',
        'min_samples_split': 3,
        'min_samples_leaf': 6,
        'random_state': 42
    }


    gb_model = GradientBoostingClassifier(**best_gb_params)
    gb_model.fit(X_train, y_train)

    # --- Prepare data for visualization ---

    # 1. Take a sample of 20 nurses (from the test set)
    sample_size = 20
    X_sample = X_test.sample(sample_size, random_state=42)
    y_sample = y_test.loc[X_sample.index]  # Use .loc to avoid SettingWithCopyWarning

    # 2. Get the model's predictions for the sample
    y_pred_sample = gb_model.predict(X_sample)

    # 3. Create a DataFrame for visualization
    plot_df = pd.DataFrame({
        'Index': X_sample.index,  # Use original indices
        'Real': y_sample,
        'Predicted': y_pred_sample
    })

    # 4. Convert to "long" format to facilitate plotting with Seaborn
    plot_df_long = plot_df.melt(id_vars='Index', value_vars=['Real', 'Predicted'],
                                var_name='Type', value_name='Anxiety Level')

    # --- Create the plot ---
    plt.figure(figsize=(14, 8))
    sns.barplot(x='Index', y='Anxiety Level', hue='Type', data=plot_df_long,
                palette={"Real": "skyblue", "Predicted": "salmon"})

    plt.title('Comparison of Real vs. Predicted Anxiety Levels (Gradient Boosting) - Sample of 20 Nurses')
    plt.xlabel('Nurse Index (Sample)')
    plt.ylabel('Anxiety Level')
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.tight_layout()
    plt.legend(title='Type')
    plt.show()

    # --- (Optional) Print the comparison table ---
    print("\nComparison Table (Real vs. Predicted):")
    print(plot_df)

Using k_neighbors = 5 for SMOTE.
Class distribution before SMOTE: Counter({0: 62, 1: 49, 2: 22, 3: 7})
Class distribution after SMOTE: Counter({1: 100, 0: 100, 2: 100, 3: 100})

Comparison Table (Real vs. Predicted):
     Index  Real  Predicted
170    170     0          0
259    259     2          2
262    262     2          2
224    224     1          1
5        5     2          1
119    119     0          0
9        9     1          1
276    276     2          2
275    275     2          2
360    360     3          3
358    358     3          2
261    261     2          2
318    318     3          3
326    326     3          3
81      81     0          0
329    329     3          3
226    226     1          1
12      12     0          1
101    101     2          0
162    162     0          0
